In [ ]:
import torch

In [4]:
sample_data = torch.arange(10, dtype=torch.float).view(1,-1)

In [1]:
from model import WaveNet
import torch.nn as nn

In [6]:
dialted_conv = DialatedConv1d(1,1,3,2)
filter_conv = nn.Conv1d(1,1,3)
gated_conv = nn.Conv1d(1,1,3)

In [7]:
x = dialted_conv(sample_data)

In [8]:
tanh_act = torch.tanh_(filter_conv(x))
sigmoid_act = torch.sigmoid_(gated_conv(x))

tanh_act, sigmoid_act

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


(tensor([[-0.5480, -0.6134, -0.6713, -0.7221]], grad_fn=<AsStridedBackward0>),
 tensor([[0.4493, 0.4181, 0.3875, 0.3578]], grad_fn=<AsStridedBackward0>))

In [11]:
z = tanh_act * sigmoid_act
z

tensor([[-0.2462, -0.2564, -0.2601, -0.2584]], grad_fn=<MulBackward0>)

The figure of the residual block along with the equation makes it seems like there are 
two convolutions for both gated and filter but in the figure there is just a dialted convolution
and you feed the output of the two into the non linearities so with that in mind this is the correction of
the above code

This is the equation for reference that caused the missconception:

z = tanh (Wf,k ∗ x) σ (Wg,k ∗ x), where W is a convolutional filter

*Note: I would need to test later if this makes a difference
*Note: Maybe instead of a gated using conv it is gated using linear unit

In [13]:
tanh_act = torch.tanh_(x)
sigmoid_act = torch.sigmoid_(x)

tanh_act, sigmoid_act

(tensor([[0.6385, 0.6698, 0.6914, 0.7058, 0.7151, 0.7210]],
        grad_fn=<AsStridedBackward0>),
 tensor([[0.6385, 0.6698, 0.6914, 0.7058, 0.7151, 0.7210]],
        grad_fn=<AsStridedBackward0>))

In [14]:
z = tanh_act * sigmoid_act
z

tensor([[0.4077, 0.4487, 0.4781, 0.4982, 0.5114, 0.5199]],
       grad_fn=<MulBackward0>)

Next we have to create a 1x1 convolution filter for residual channel and skip channels in the image there is only one but there might be neccesary that we need two seperate ones this is based on looking at other code repos

From looking at some repos it seems when they name variables like res channels and skip channels they mean the output size of those convolution operations